In [1]:
import numpy as np
import face_recognition as fr
import cv2
import os
import glob

In [2]:
video_capture=cv2.VideoCapture(0)

In [3]:
prem_image = fr.load_image_file("prem.jpg")
prem_face_encoding = fr.face_encodings(prem_image)[0]
jitendra_image = fr.load_image_file("jitendra.jpg")
jitendra_face_encoding = fr.face_encodings(jitendra_image)[0]
gunu_image = fr.load_image_file("gunu.jpg")
gunu_face_encoding = fr.face_encodings(gunu_image)[0]
known_face_encodings=[prem_face_encoding,jitendra_face_encoding,gunu_face_encoding]
known_face_names=["prem","jitendra","Divya"]
p='/home/prem/Desktop/ml-kiit'
dirname = os.path.dirname(p)
path = os.path.join(dirname,'known_people/')

In [4]:
list_of_files=[f for f in glob.glob(path+'*.jpg')]

In [5]:
number_files=len(list_of_files)

In [6]:
names=list_of_files.copy()

In [7]:
for i in range(number_files):
    globals()['image_{}'.format(i)]=fr.load_image_file(list_of_files[i])
    globals()['image_encoding_{}'.format(i)]=fr.face_encodings(globals()['image_{}'.format(i)])[0]
    known_face_encodings.append(globals()['image_encoding_{}'.format(i)])
    names[i]=names[i].replace("known_people/","")
    known_face_names.append(names[i])

In [8]:
face_locations=[]
face_encodings=[]
face_names=[]
process_this_frame=True

In [9]:
while True:
    ret, frame = video_capture.read()
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    if process_this_frame:
        face_locations = fr.face_locations(rgb_small_frame)
        face_encodings = fr.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            matches = fr.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            face_distances = fr.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            face_names.append(name)
    process_this_frame = not process_this_frame
    for(top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Hit 'q' on the keyboard to quit!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [10]:
video_capture.release()
cv2.destroyAllWindows()